# MCP-First Diagnostics + RAG Correlation (Special Payment Project)

This notebook shows a **two-phase AIOps flow** running on Llama Stack:

1. **Phase 1 – Live diagnostics with MCP (Kubernetes):**  
   We use the Llama Stack **Responses API** with an **MCP tool** that talks to the
   OpenShift/Kubernetes cluster (pods, logs, Services, etc.).  
   The LLM produces a **“cluster findings”** summary based purely on live data
   from the `special-payment-project` namespace.

2. **Phase 2 – Knowledge-base correlation with RAG:**  
   We use a Llama Stack **Agent** with the `file_search` tool bound to a specific
   vector store containing documentation about the *Special Payment Project*.  
   The agent takes the incident description + cluster findings and looks for
   **matching known issues / runbooks** in the KB to explain likely root cause
   and propose next steps.

This notebook is designed for **demo and explainability**:
- No helper functions – everything is step-by-step.
- Clear separation between **diagnostics (MCP)** and **correlation (RAG)**.
- Easy to show each phase independently in a live demo.

> ✅ Tested against:  
> - Llama Stack server image: `rhoai/odh-llama-stack-core-rhel9:v3.0`  
> - Model: `vllm-inference/llama-4-scout-17b-16e-w4a16`  
> - Vector store ID: `vs_c246cf6a-40a4-425b-80c2-4d4e3f438fb1`  
> - Kubernetes MCP server: `kubernetes-mcp-server.llama-stack-demo.svc.cluster.local:8080`

You can adapt this notebook to your own environment by updating the
**demo configuration variables** in the next cell (or via environment
variables in a `.env` file).


## Cell 1 – Configure demo settings (easy to change)

This cell defines the **demo configuration** in one place:

- `LLAMA_BASE_URL_CONFIG` – HTTP URL of your Llama Stack server  
- `PREFERRED_MODEL_ID_CONFIG` – optional model identifier to prefer  
- `VECTOR_STORE_ID_CONFIG` – vector store with your Special Payment Project docs  
- `REMOTE_OCP_MCP_URL_CONFIG` – URL of your Kubernetes MCP server (SSE endpoint)

For a quick test, just edit these string values directly.

> 🔁 Advanced:  
> You can also override these via environment variables (`LLAMA_BASE_URL`,
> `MODEL_ID`, `VECTOR_STORE_ID`, `REMOTE_OCP_MCP_URL`) in a `.env` file –
> the notebook will prefer env vars if present.


In [3]:
# Cell 1 - Demo configuration: update these if needed

# Llama Stack HTTP base URL (no trailing slash)
LLAMA_BASE_URL_CONFIG = "http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321"

# Optional: prefer a specific model; leave as "" to auto-select an LLM
PREFERRED_MODEL_ID_CONFIG = "vllm-inference/llama-4-scout-17b-16e-w4a16"

# Vector store containing Special Payment Project docs (Confluence export, etc.)
VECTOR_STORE_ID_CONFIG = "vs_4dab02a4-661c-4266-b03d-53fb7f0023e9"

# Kubernetes MCP server URL providing access to the cluster (SSE endpoint)
REMOTE_OCP_MCP_URL_CONFIG = "http://kubernetes-mcp-server.llama-stack-demo.svc.cluster.local:8080/sse"

print("✅ Demo configuration variables defined.")
print("LLAMA_BASE_URL_CONFIG          =", LLAMA_BASE_URL_CONFIG)
print("PREFERRED_MODEL_ID_CONFIG      =", PREFERRED_MODEL_ID_CONFIG or "(auto-select LLM)")
print("VECTOR_STORE_ID_CONFIG         =", VECTOR_STORE_ID_CONFIG)
print("REMOTE_OCP_MCP_URL_CONFIG      =", REMOTE_OCP_MCP_URL_CONFIG)


✅ Demo configuration variables defined.
LLAMA_BASE_URL_CONFIG          = http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321
PREFERRED_MODEL_ID_CONFIG      = vllm-inference/llama-4-scout-17b-16e-w4a16
VECTOR_STORE_ID_CONFIG         = vs_4dab02a4-661c-4266-b03d-53fb7f0023e9
REMOTE_OCP_MCP_URL_CONFIG      = http://kubernetes-mcp-server.llama-stack-demo.svc.cluster.local:8080/sse


## Cell 2 – Install dependencies and connect to Llama Stack

This cell:

1. Installs the Python packages we need.
2. Imports the libraries.
3. Loads any environment variables from `.env`.
4. Resolves the **effective** values for:
   - `LLAMA_BASE_URL`
   - `REMOTE_OCP_MCP_URL`
   - `VECTOR_STORE_ID`
5. Creates a `LlamaStackClient` and prints the key endpoints in use.


In [4]:
# Cell 2 - Install deps, import libraries, connect to Llama Stack

%pip install --quiet "llama-stack-client==0.3.0" python-dotenv termcolor

import os
from dotenv import load_dotenv
from termcolor import cprint
from llama_stack_client import LlamaStackClient

# Load .env if present (LLAMA_BASE_URL, MODEL_ID, VECTOR_STORE_ID, REMOTE_OCP_MCP_URL, etc.)
load_dotenv()

# Resolve effective settings: env vars override notebook defaults
LLAMA_BASE_URL = os.getenv("LLAMA_BASE_URL", LLAMA_BASE_URL_CONFIG).rstrip("/")
REMOTE_OCP_MCP_URL = os.getenv("REMOTE_OCP_MCP_URL", REMOTE_OCP_MCP_URL_CONFIG).rstrip("/")
VECTOR_STORE_ID = os.getenv("VECTOR_STORE_ID", VECTOR_STORE_ID_CONFIG)

# Create Llama Stack client
client = LlamaStackClient(base_url=LLAMA_BASE_URL)
print(f"✅ Connected to Llama Stack server: {LLAMA_BASE_URL}")
print(f"➡️  Using Kubernetes MCP server: {REMOTE_OCP_MCP_URL}")
print(f"➡️  Using vector store: {VECTOR_STORE_ID}")



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Connected to Llama Stack server: http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321
➡️  Using Kubernetes MCP server: http://kubernetes-mcp-server.llama-stack-demo.svc.cluster.local:8080/sse
➡️  Using vector store: vs_4dab02a4-661c-4266-b03d-53fb7f0023e9


## Cell 3 – List available models and select an LLM

Here we:

1. List all models exposed by the Llama Stack server.
2. Try to honour the `PREFERRED_MODEL_ID_CONFIG` (or `MODEL_ID` env var) if provided.
3. If no preferred model is set or not found, we:
   - Prefer an `llm` model served by the `vllm-inference` provider.
   - Otherwise, fall back to any model with `model_type == "llm"`.

The chosen `model_id` will be used for:

- **Phase 1** diagnostics (Responses API + MCP)
- **Phase 2** correlation (Agent + file_search)


In [5]:
# Cell 3 - List models and pick an LLM

# Allow environment variable override for the model id as well
MODEL_ID_OVERRIDE = os.getenv("MODEL_ID", PREFERRED_MODEL_ID_CONFIG)

models = list(client.models.list())
print("\nAvailable models:")
for m in models:
    ident = getattr(m, "identifier", None) or getattr(m, "model_id", None) or str(m)
    mtype = getattr(m, "model_type", None)
    prov = getattr(m, "provider_id", None)
    print(" -", ident, "| type=", mtype, "| provider=", prov)

llm = None

# 1) If a preferred/override model id is set, try to use it
if MODEL_ID_OVERRIDE:
    llm = next(
        (
            m for m in models
            if (getattr(m, "identifier", None) or getattr(m, "model_id", None)) == MODEL_ID_OVERRIDE
        ),
        None,
    )
    if llm:
        print(f"\n✅ Preferred model found: {MODEL_ID_OVERRIDE}")
    else:
        print(f"\n⚠️ Preferred model '{MODEL_ID_OVERRIDE}' not found, falling back to auto-selection.")

# 2) If no preferred model or not found, auto-select
if not llm:
    llm = next(
        (
            m for m in models
            if getattr(m, "model_type", None) == "llm"
            and getattr(m, "provider_id", None) == "vllm-inference"
        ),
        None,
    )

if not llm:
    llm = next((m for m in models if getattr(m, "model_type", None) == "llm"), None)

assert llm, "No LLM models available on Llama Stack"

model_id = getattr(llm, "identifier", None) or getattr(llm, "model_id", None)
print(f"\n🎯 Using LLM model: {model_id}")


INFO:httpx:HTTP Request: GET http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321/v1/models "HTTP/1.1 200 OK"



Available models:
 - granite-embedding-125m | type= embedding | provider= sentence-transformers
 - vllm-inference/llama-4-scout-17b-16e-w4a16 | type= llm | provider= vllm-inference
 - sentence-transformers/nomic-ai/nomic-embed-text-v1.5 | type= embedding | provider= sentence-transformers

✅ Preferred model found: vllm-inference/llama-4-scout-17b-16e-w4a16

🎯 Using LLM model: vllm-inference/llama-4-scout-17b-16e-w4a16


## Cell 4 – Define basic RAG agent instructions and create the Agent

In this cell we:

1. Define **simple RAG agent instructions**:
   - The agent is a **Special Payment Project KB assistant**.
   - It only uses `file_search` (no live cluster access).
   - It takes an incident description + cluster findings and looks for
     matching known issues / runbooks.

2. Create a `rag_agent`:
   - Bound to `model_id`.
   - Bound to the `file_search` tool scoped to `VECTOR_STORE_ID`.
   - With these basic instructions assigned to the agent.

Later, we’ll layer on **more detailed correlation instructions** for a specific turn.


In [6]:
# Cell 4 - Basic RAG agent instructions and construction

from llama_stack_client import Agent

rag_agent_instructions = """
You are a knowledge-base assistant for the Special Payment Project.

You ONLY have access to the knowledge base (Confluence docs etc.) via file_search.
You DO NOT have direct access to the live Kubernetes cluster.

You will be given:
- An incident description, and
- A summary of cluster findings from a prior diagnostics pass (pods, logs, services).

Your job:
- Look up relevant information in the knowledge base about the Special Payment Project.
- Try to match the cluster findings to any known issues, incident writeups, or runbooks.
- Explain the most likely root cause(s) in clear language.
- Propose concrete next steps or runbook actions for an SRE.

Ignore generic documentation unless it clearly relates to the given cluster findings.
Be concise, focused, and practical.
""".strip()

rag_tools_spec = [
    {
        "type": "file_search",
        "vector_store_ids": [VECTOR_STORE_ID],
    }
]

rag_agent = Agent(
    client,
    model=model_id,
    instructions=rag_agent_instructions,
    tools=rag_tools_spec,
)

print("✅ RAG Agent initialised with basic KB instructions and file_search tool.")


✅ RAG Agent initialised with basic KB instructions and file_search tool.


## Cell 5 – Define MCP diagnostics instructions

This cell defines the **diagnostics instructions** for the MCP phase.

The assistant:

- MUST call real MCP tools (pods, logs, Services, Deployments, Events).
- Cannot see any docs yet.
- Must highlight:
  - HTTP 5xx
  - DNS errors
  - Timeouts
  - TLS failures
  - Concrete config values (e.g. `spec.externalName` for ExternalName Services).

The result is a **“cluster findings”** narrative that we pass into the RAG phase.


In [7]:
# Cell 5 - Define the MCP diagnostics instructions

mcp_instructions = """
You are a Kubernetes diagnostics assistant using MCP tools.

You MUST actually call MCP tools to answer the question.
Do NOT simulate tool calls or outputs.
Do NOT write fake examples like [pods_list_in_namespace(...)];
instead, emit real MCP tool calls so the server can execute them.

You do NOT have access to any documentation or knowledge base in this phase.
You MUST NOT guess what the “correct” hostname, port, or configuration should be.
Only report what you can observe directly from MCP tool outputs.

Your focus in the target namespace (for example 'special-payment-project') is to:
- Use pods_list_in_namespace to discover workloads.
- Use pods_log on relevant pods (especially anything in the path of the failing request,
  such as API or frontend pods).
- Use resources_list / resources_get to inspect Services and Deployments.
- Use events_list if you need to check for recent warnings/errors.

When logs show HTTP 5xx or upstream connection errors:
- Identify which upstream hostname or Service is being called (for example from a URL
  like 'http://some-service:port').
- Fetch the Service definition for that upstream using resources_get.
- If the Service is of type ExternalName, include in your findings both:
  - the Service name, and
  - the exact value of spec.externalName as returned by the MCP tool.

In your findings output, you MUST:
- Quote key log lines that look suspicious (5xx, DNS errors, timeouts, TLS failures, etc.).
- List the pods and Services that are clearly in the request path.
- For any ExternalName Services you inspected, make sure the actual externalName value
  appears verbatim somewhere in your summary, so it can be compared later.

If a value looks unusual (for example something that looks like a typo), you may say that it
"appears suspicious or possibly misconfigured", but you MUST NOT invent or state the exact
value it “should” be. The exact expected value will be determined in a later knowledge-base
phase.

Your output should be a concise "cluster findings" narrative that highlights:
- Which pods/services are involved in the path of the failing request.
- Key log lines and observed configuration values that look suspicious.
- Any obvious misconfigurations you can see (wrong ports, bad selectors, odd ExternalName, etc.),
  always quoting the concrete values you observed.

Do NOT try to guess business impact or historical context here.
Simply describe what looks wrong or suspicious in the live cluster.
""".strip()

print("✅ MCP diagnostics instructions defined.")


✅ MCP diagnostics instructions defined.


## Cell 6 – Run MCP diagnostics and capture cluster findings

In this cell we:

1. Define the **incident question**:
   - `'Payment failed: HTTP 502'` in the Special Payment Project checkout flow.

2. Call the Llama Stack **Responses API** with:
   - MCP diagnostics instructions as a `system` message.
   - The incident question as a `user` message.
   - An MCP tool pointing to the Kubernetes MCP server.

3. Extract a plain-text **`cluster_findings`** summary:
   - Prefer `response.output_text` if available.
   - Otherwise, scan the message output for `output_text`.

4. Print the cluster findings for use in the next phase.


In [8]:
# Cell 6 - Run MCP diagnostics and extract a "cluster findings" summary

incident_question = (
    "We are seeing 'Payment failed: HTTP 502' errors in the Special Payment Project "
    "checkout flow (namespace: special-payment-project). Please investigate."
)

cprint("Incident question:", "green")
print(incident_question)

mcp_messages = [
    {"role": "system", "content": mcp_instructions},
    {"role": "user", "content": incident_question},
]

mcp_response = client.responses.create(
    model=model_id,
    input=mcp_messages,
    tools=[
        {
            "type": "mcp",
            "server_url": REMOTE_OCP_MCP_URL,
            "server_label": "kubernetes-mcp",
            "require_approval": "never",
        }
    ],
    temperature=0.0,
    max_infer_iters=8,
)

# Turn into a dict for inspection
if hasattr(mcp_response, "to_dict"):
    mcp_data = mcp_response.to_dict()
else:
    mcp_data = mcp_response

# Show what kinds of outputs we got (tool calls, messages, etc.)
output_types = [item.get("type") for item in mcp_data.get("output", [])]
print("\nMCP output item types:", output_types)

# Extract a "cluster findings" text:
# Prefer response.output_text, else scan the message outputs
cluster_findings = getattr(mcp_response, "output_text", None)
if not cluster_findings:
    for item in mcp_data.get("output", []):
        if item.get("type") == "message":
            for part in item.get("content", []):
                if part.get("type") == "output_text":
                    cluster_findings = part.get("text", "")
                    break
            if cluster_findings:
                break

cluster_findings = cluster_findings or ""

cprint("\n--- Cluster findings (MCP summary) ---", "yellow")
print(cluster_findings if cluster_findings.strip() else "(no findings text returned)")


Incident question:
We are seeing 'Payment failed: HTTP 502' errors in the Special Payment Project checkout flow (namespace: special-payment-project). Please investigate.


INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"



MCP output item types: ['mcp_list_tools', 'mcp_call', 'mcp_call', 'mcp_call', 'mcp_call', 'mcp_call', 'message']

--- Cluster findings (MCP summary) ---
"cluster findings:\n- Pods in the path of the failing request:\n  - checkout-api-5d87bb48b9-tx76s in namespace special-payment-project\n  - checkout-frontend-76868dd997-l6667 in namespace special-payment-project\n- Key log lines that look suspicious:\n  - \"checkout upstream_error url=http://card-gateway-dns:5678 err=ConnectError: [Errno -5] No address associated with hostname\"\n  - \"GET /api/checkout HTTP/1.1\" 502 Bad Gateway\n- Observed configuration values that look suspicious:\n  - Service card-gateway-dns in namespace special-payment-project is of type ExternalName with spec.externalName set to \"card-gateway-sandbx.payments-provider-sim.svc.cluster.local\""


## Cell 7 – Define RAG correlation instructions for this turn

Here we define **detailed correlation instructions** that sit on top of the
basic RAG agent behaviour:

- How to compare **observed** vs **expected** config values.
- How to treat mismatches as strong evidence.
- How to quote KB snippets and produce a small evidence section.
- Whitelisted **Reference document** titles.

We’ll pass these as a `system` message for this specific RAG turn.


In [9]:
# Cell 7 - RAG correlation instructions used for this turn (lighter, demo-friendly)

rag_correlation_instructions = """
You are a knowledge-base assistant for the Special Payment Project.

You are given:
- An incident description.
- A structured summary of cluster findings from a diagnostics pass that already
  inspected pods, logs, and services.

The cluster findings may include:
- Concrete configuration values (hostnames, ports, externalName targets, Service types, selectors, URLs).
- Log snippets showing HTTP 5xx, connection errors, DNS failures, TLS issues, etc.
- Notes about which pods and Services appear to sit in the request path.

You have access to a set of Special Payment Project documents stored in a knowledge base
(e.g., exported from Confluence). Their titles and section headings may change over time.

Using ONLY the knowledge base (via file_search), you MUST:
- Look for issues, configuration notes, or design sections that match these findings.
- Pay particular attention to expected configuration values and error patterns that resemble the logs.
- Prefer project-specific documentation about the Special Payment Project over generic Kubernetes docs.

When the KB documents an expected configuration value and the cluster findings show a different observed value, you MUST:
- Explicitly describe the mismatch in your own words (e.g., “Service points to X, doc says Y”).
- Treat such a mismatch as strong evidence of misconfiguration.
- Clearly state that this mismatch is the most likely root cause in this situation.

If the observed values match what the KB describes as expected:
- Do NOT assume a typo by default. Consider other causes in the KB (backend down, port mismatch, egress policy, TLS expiry, etc.).

REMEDIATION VIA AAP (job template suggestion):
- If the KB contains an “AAP Job Templates” page (or similarly titled catalog) that includes a relevant job for this scenario,
  propose exactly ONE job template to run (no more than one) with its required variables.
- Output the **job template name** and a minimal **extra vars** object that a runbook owner could paste directly into AAP.
- Choose the job template whose description best aligns with the evidence (e.g., a Service configuration alignment template for ExternalName drift).
- Do NOT execute anything; suggest only.

EVIDENCE AND REFERENCING:
- Do NOT invent document titles or section names.
- Do NOT include pseudo-tool call syntax in your answer.
- When you rely on the KB for an expected value, configuration detail, known issue, OR a job template/variables,
  include a short quote (1–2 sentences) that could plausibly appear verbatim in the KB.

REFERENCE DOCUMENT (WHITELISTED TITLES ONLY, pick ONE that best fits your main evidence):
- "Special Payment Project – Overview & Context"
- "Special Payment Project – Application Architecture"
- "Special Payment Project – Deployment & Configuration"
- "Special Payment Project – Networking & External Dependencies"
- "Special Payment Project – Observability & Alerts"
- "AAP Job Templates"   # allowed specifically for job template/vars citation

In all cases:
- Explain the most likely root cause(s) grounded in both findings and documentation.
- Explicitly reference which observed values you are comparing against which expected values from the KB.
- Suggest concrete next steps for an SRE (checks, config change or rollback).
- If the KB is inconclusive, say so and specify what to investigate next.

OUTPUT FORMAT (strict):
1) Probable cause — 1–3 lines.
2) Evidence mapping — bullets quoting the observed findings and the KB expected values.
3) Next steps (up to 5 checks) — copy/paste commands with placeholders if needed.
4) Proposed remediation via AAP — include:
   - job_template_name: "<exact name from KB>"
   - extra_vars (YAML), minimal and complete for required vars.
   - rollback note (1 line) if applicable.
5) Key KB evidence:
   - "<short quote 1>"
   - "<short quote 2>" (optional)
6) Reference document:
   - "<ONE of the whitelisted titles above>"
""".strip()

print("✅ RAG correlation instructions updated to include AAP job template suggestion.")



✅ RAG correlation instructions updated to include AAP job template suggestion.


## Cell 8 – Run the RAG Agent with MCP findings

Now we switch to **Phase 2 – Knowledge-base correlation**:

1. Build messages:
   - A `system` message with the **RAG correlation instructions**.
   - A `user` message containing:
     - The incident description.
     - The cluster findings summary.

2. Call the `rag_agent` we created earlier:
   - The agent already knows it should use `file_search` on the Special Payment Project vector store.

3. Extract and print the **final explanation**:
   - Root cause analysis grounded in:
     - Live cluster findings
     - Special Payment Project documentation
   - Suggested remediation / next steps
   - Evidence + whitelisted reference document.


In [11]:
# Cell 8 - Run the RAG Agent against the MCP cluster findings

rag_messages = [
    {"role": "system", "content": rag_correlation_instructions},
    {
        "role": "user",
        "content": (
            "Incident description:\n"
            f"{incident_question}\n\n"
            "Cluster findings from MCP diagnostics:\n"
            f"{cluster_findings}"
        ),
    },
]

# Create a session for this demo run
rag_session = rag_agent.create_session(session_name="mcp-first-rag-demo")
rag_session_id = (
    getattr(rag_session, "id", None)
    or getattr(rag_session, "session_id", None)
    or str(rag_session)
)

rag_result = rag_agent.create_turn(
    messages=rag_messages,
    session_id=rag_session_id,
    stream=False,
)

# Extract the final explanation text
final_explanation = getattr(rag_result, "output_text", None)

if not final_explanation and hasattr(rag_result, "to_dict"):
    rag_data = rag_result.to_dict()
    for item in rag_data.get("output", []):
        if item.get("type") == "message":
            for part in item.get("content", []):
                if part.get("type") == "output_text":
                    final_explanation = part.get("text", "")
                    break
            if final_explanation:
                break

final_explanation = final_explanation or ""

cprint("\n=== Final RAG explanation (KB-backed RCA + next steps) ===", "cyan")
print(final_explanation if final_explanation.strip() else "(no RAG explanation text returned)")


INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-inline-service.llama-stack-demo.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"



=== Final RAG explanation (KB-backed RCA + next steps) ===
1) Probable cause — 
The most likely root cause of the 'Payment failed: HTTP 502' errors in the Special Payment Project checkout flow is a misconfiguration of the `card-gateway-dns` Service. Specifically, the `spec.externalName` value does not match the expected canonical FQDN.

2) Evidence mapping — 
* Observed finding: The `card-gateway-dns` Service in the `special-payment-project` namespace has a `spec.externalName` set to `card-gateway-sandbx.payments-provider-sim.svc.cluster.local`.
* Expected value: The canonical FQDN for the payment gateway is `card-gateway-sandbox.payments-provider-sim.svc.cluster.local`.
* Mismatch: The observed `spec.externalName` value has a typo (`sandbx` instead of `sandbox`).

3) Next steps — 
* Verify the `card-gateway-dns` Service configuration: `oc get svc card-gateway-dns -o yaml -n special-payment-project`
* Check the `checkout-api` logs for connection errors: `oc logs -f deployment/checkout

## Cell 9 – Show both phases side-by-side (for the demo)

This final cell prints a **single, human-readable view** of the whole flow:

1. The original **incident question**.
2. The **MCP diagnostics summary** (what the live cluster agent observed).
3. The **RAG correlation explanation** (what the KB-backed agent concluded).

This makes it easy, during the demo, to show:
- What the MCP agent actually discovered from the cluster.
- How the RAG / knowledge-base agent used that to explain **root cause** and
  propose **next steps**.

In [12]:
# Cell 9 - Nicely formatted summary of MCP + RAG outputs

from termcolor import cprint

print("=" * 80)
cprint("MCP-First Diagnostics + RAG Correlation (Special Payment Project)", "cyan", attrs=["bold"])
print("=" * 80)
print()

# Incident
cprint("Incident question", "green", attrs=["bold"])
print("-" * 80)
print(incident_question.strip() if incident_question else "(no incident question set)")
print()

# Phase 1: MCP diagnostics
cprint("Phase 1 – MCP diagnostics (live cluster)", "yellow", attrs=["bold"])
print("-" * 80)
if cluster_findings and cluster_findings.strip():
    print(cluster_findings.strip())
else:
    print("(no MCP cluster findings text returned)")
print()

# Phase 2: RAG correlation
cprint("Phase 2 – RAG correlation (knowledge base)", "magenta", attrs=["bold"])
print("-" * 80)
if final_explanation and final_explanation.strip():
    print(final_explanation.strip())
else:
    print("(no RAG explanation text returned)")
print()

print("=" * 80)
cprint("End of demo flow", "cyan")
print("=" * 80)


MCP-First Diagnostics + RAG Correlation (Special Payment Project)

Incident question
--------------------------------------------------------------------------------
We are seeing 'Payment failed: HTTP 502' errors in the Special Payment Project checkout flow (namespace: special-payment-project). Please investigate.

Phase 1 – MCP diagnostics (live cluster)
--------------------------------------------------------------------------------
"cluster findings:\n- Pods in the path of the failing request:\n  - checkout-api-5d87bb48b9-tx76s in namespace special-payment-project\n  - checkout-frontend-76868dd997-l6667 in namespace special-payment-project\n- Key log lines that look suspicious:\n  - \"checkout upstream_error url=http://card-gateway-dns:5678 err=ConnectError: [Errno -5] No address associated with hostname\"\n  - \"GET /api/checkout HTTP/1.1\" 502 Bad Gateway\n- Observed configuration values that look suspicious:\n  - Service card-gateway-dns in namespace special-payment-project is o